In [ ]:
!pip install jsonlines
!pip install pymorphy2

In [19]:
import pandas as pd
import jsonlines
import string
import re

import glob
import io
import os
import random

import pymorphy2
m = pymorphy2.MorphAnalyzer()

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import string
from string import punctuation

from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ekostrykina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def get_data(filename):
    data = []
    with jsonlines.open(f'TERRa/{filename}.jsonl') as reader:
        for obj in reader:
            data.append(obj)
    return pd.DataFrame(data)

In [4]:
val_df = get_data('val')

In [5]:
val_df

,premise,hypothesis,label,idx
0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,entailment,0
1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,entailment,1
2,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,entailment,2
3,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,entailment,3
4,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,entailment,4
...,...,...,...,...
302,"Грабитель поджидал своих жертв и, угрожая им п...",У грабителя был пистолет.,entailment,302
303,Во время обыска в квартире подозреваемого поли...,С ноября прошлого года подозреваемый был на св...,entailment,303
304,"""""""После инцидента 15 августа 2011 г. ко мне п...",Береговой не сделал видеозапись.,not_entailment,304
305,Сыграло ли это событие роль в решении обвиняем...,Дивенни мог работать на иностранную разведку.,entailment,305


In [7]:
prepositions = 'за в во на с за о об под по про при через без близ вместо вне возле для до из из-за из-под кроме мимо около от после ради с у к ко над перед между'.split()

In [8]:
prepositions

['за',
 'в',
 'во',
 'на',
 'с',
 'за',
 'о',
 'об',
 'под',
 'по',
 'про',
 'при',
 'через',
 'без',
 'близ',
 'вместо',
 'вне',
 'возле',
 'для',
 'до',
 'из',
 'из-за',
 'из-под',
 'кроме',
 'мимо',
 'около',
 'от',
 'после',
 'ради',
 'с',
 'у',
 'к',
 'ко',
 'над',
 'перед',
 'между']

In [20]:
def change_preposition(sentence):
    text = word_tokenize(sentence)
    for idx, word in enumerate(text):
        parsed_word = m.parse(word)[0]
        if parsed_word.tag.POS == 'PREP':
            new_word = random.choice(prepositions)
            while word.lower() == new_word:
                new_word = random.choice(prepositions)
            text[idx] = new_word
            text = ' '.join(text)
            # удаляем лишние пробелы
            pat = "\s+([{}]+)".format(re.escape(string.punctuation))
            output_text = re.sub("\s{2,}", " ", re.sub(pat, r"\1", text))
            return output_text, True, word, new_word
    return sentence, False, None, None

In [25]:
change_preposition('На улице Кеэвисе жителей эвакуировали из-за мины.')

('во улице Кеэвисе жителей эвакуировали из-за мины.', True, 'На', 'во')

In [27]:
attacked_data = []
flags = []
original_words = []
inflected_words = []

for i in tqdm(val_df['hypothesis']):
    sentence, flag, original_word, inflected_word = change_preposition(i)
    attacked_data.append(sentence)
    flags.append(flag)
    original_words.append(original_word)
    inflected_words.append(inflected_word)

  0%|          | 0/307 [00:00<?, ?it/s]

In [29]:
val_df['original_hypothesis'] = val_df['hypothesis']
val_df['hypothesis'] = attacked_data
val_df['flags'] = flags
val_df['original_words'] = original_words
val_df['inflected_words'] = inflected_words

In [30]:
val_df

,premise,hypothesis,label,idx,original_hypothesis,flags,original_words,inflected_words
0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,entailment,0,Взыскать компенсацию не получится.,False,None,None
1,"Об этом вечером во вторник, 17 января, сообщил...",у улице Кеэвисе жителей эвакуировали из-за мины.,entailment,1,На улице Кеэвисе жителей эвакуировали из-за мины.,True,На,у
2,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,entailment,2,81-летний пенсионер застрелил обидчика дочери.,False,None,None
3,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил из-за снегу контур человеческ...,entailment,3,Кто-то изобразил на снегу контур человеческого...,True,на,из-за
4,"Также установлено, что работники учреждения не...",из-под детском саду не все вовремя проходят ме...,entailment,4,В детском саду не все вовремя проходят медосмотр.,True,В,из-под
...,...,...,...,...,...,...,...,...
302,"Грабитель поджидал своих жертв и, угрожая им п...",об грабителя был пистолет.,entailment,302,У грабителя был пистолет.,True,У,об
303,Во время обыска в квартире подозреваемого поли...,у ноября прошлого года подозреваемый был на св...,entailment,303,С ноября прошлого года подозреваемый был на св...,True,С,у
304,"""""""После инцидента 15 августа 2011 г. ко мне п...",Береговой не сделал видеозапись.,not_entailment,304,Береговой не сделал видеозапись.,False,None,None
305,Сыграло ли это событие роль в решении обвиняем...,Дивенни мог работать ради иностранную разведку.,entailment,305,Дивенни мог работать на иностранную разведку.,True,на,ради


In [31]:
val_df.to_csv('attack3_1_val.csv', index=False)